<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/AI/Mod%C3%A8le000001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fonctions de base

In [397]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TIPE'
import os
from tensorflow.python.client import device_lib
print("Utilise le",str(device_lib.list_local_devices()[0])[15:18])
print()
print()
from google.colab import files
import tensorflow as tf
from tensorflow.python import debug as tf_debug

from tensorflow.keras.layers import Layer
import matplotlib.gridspec as gridspec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,concatenate,Subtract,Multiply,Average, Concatenate,Reshape, Add, BatchNormalization, Conv2D
from tensorflow.keras.layers import Reshape,Lambda
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Convolution2D, AveragePooling2D,MaxPooling2D
import tensorflow.keras.losses
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K
import numpy as np
import scipy
from PIL import Image
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import cv2
import matplotlib.pyplot as plt
images = ["Galaxies_resized/"+f for f in os.listdir("Galaxies_resized/")]
noises = []

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TIPE
Utilise le CPU




In [0]:
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Optimizer


# Ported from https://github.com/LiyuanLucasLiu/RAdam/blob/master/radam.py
class RectifiedAdam(Optimizer):
    """RectifiedAdam optimizer.
    Default parameters follow those provided in the original paper.
    # Arguments
        lr: float >= 0. Learning rate.
        final_lr: float >= 0. Final learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        gamma: float >= 0. Convergence speed of the bound function.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: Weight decay weight.
        amsbound: boolean. Whether to apply the AMSBound variant of this
            algorithm.
    # References
        - [On the Variance of the Adaptive Learning Rate and Beyond]
          (https://arxiv.org/abs/1908.03265)
        - [Adam - A Method for Stochastic Optimization]
          (https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond]
          (https://openreview.net/forum?id=ryQu7f-RZ)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0.0, **kwargs):
        super(RectifiedAdam, self).__init__(**kwargs)

        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')

        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay

        self.weight_decay = float(weight_decay)

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            beta2_t = self.beta_2 ** t
            N_sma_max = 2 / (1 - self.beta_2) - 1
            N_sma = N_sma_max - 2 * t * beta2_t / (1 - beta2_t)

            # apply weight decay
            if self.weight_decay != 0.:
                p_wd = p - self.weight_decay * lr * p
            else:
                p_wd = None

            if p_wd is None:
                p_ = p
            else:
                p_ = p_wd

            def gt_path():
                step_size = lr * K.sqrt(
                    (1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max /
                    (N_sma_max - 2)) / (1 - self.beta_1 ** t)

                denom = K.sqrt(v_t) + self.epsilon
                p_t = p_ - step_size * (m_t / denom)

                return p_t

            def lt_path():
                step_size = lr / (1 - self.beta_1 ** t)
                p_t = p_ - step_size * m_t

                return p_t

            p_t = K.switch(N_sma > 5, gt_path, lt_path)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'weight_decay': self.weight_decay}
        base_config = super(RectifiedAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
def next_batch(batch_size, images,tailleAttendue,formatArray):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    formatArray : format de sortie des données de l'image ; utiliser numpy
    """
    imageEntreeTensor = []
    imageSortieTensor = []
    while len(imageEntreeTensor) < batch_size:
        try:
            np.random.shuffle(images)#choix aléatoire de l'image
            image = cv2.imread(images[0])#Ouvre en rgb l'image nettoyée
            resizedImage = cv2.resize(image,(tailleAttendue,tailleAttendue))
            imageSortieTensor.append(np.array(resizedImage,dtype=formatArray))
            imageEntreeTensor.append(np.array(resizedImage,dtype=formatArray))
        except:
            print("Error in next_batch")
    imageEntreeTensor = np.array(imageEntreeTensor,formatArray)
    return [imageEntreeTensor,imageEntreeTensor]

In [0]:
def next_batch_bruit_voile(batch_size, images,tailleAttendue,formatArray,facteursVoile,bruitParam,plageVal=[0,255]):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    formatArray : format de sortie des données de l'image ; utiliser numpy
    facteurVoile : liste de valeur entre 0 et 1 contenant l'atténuation pour chaque couche de l'image
    bruitParam : liste avec dans l'ordre moyenne et écart type
    """
    assert plageVal[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]-bruitParam[1] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]+bruitParam[1] <= plageVal[1]
    
    imageEntreeTensor,imageSortieTensor = next_batch(batch_size,  images,tailleAttendue,formatArray)
    imageSortieTensorCopy = np.array(imageSortieTensor,dtype=np.float32)
    for image in range(imageSortieTensorCopy.shape[0]):
        for rgbIndex in range(3):
            imageSortieTensorCopy[image,:,:,rgbIndex] *= facteursVoile[rgbIndex]
    imageSortieTensorCopy = np.clip(imageSortieTensorCopy + np.random.normal(bruitParam[0],bruitParam[1],imageSortieTensorCopy.shape),plageVal[0],plageVal[1])
    return [imageEntreeTensor,np.array(imageSortieTensorCopy,dtype=formatArray)]

In [0]:
def next_batch_bruit_voile_2(batch_size, images,tailleAttendue,formatArray,facteursVoile,bruitParam,plageVal=[0,255]):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    La versison 2 fait les  modification sélectives de couleurs après avoir ajouté le bruit
    formatArray : format de sortie des données de l'image ; utiliser numpy
    facteurVoile : liste de valeur entre 0 et 1 contenant l'atténuation pour chaque couche de l'image
    bruitParam : liste avec dans l'ordre moyenne et écart type
    """
    assert plageVal[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]-bruitParam[1] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]+bruitParam[1] <= plageVal[1]
    
    imageEntreeTensor,imageSortieTensor = next_batch(batch_size,  images,tailleAttendue,formatArray)
    imageSortieTensorCopy = np.array(imageSortieTensor,dtype=np.float32)
    for image in range(imageSortieTensorCopy.shape[0]):
        for rgbIndex in range(3):
            imageSortieTensorCopy[image,:,:,rgbIndex] += np.random.normal(bruitParam[0],bruitParam[1])
            imageSortieTensorCopy[image,:,:,rgbIndex] *= facteursVoile[rgbIndex]
    imageSortieTensorCopy /= np.max(imageSortieTensorCopy)
    imageSortieTensorCopy *= plageVal[1]
    imageSortieTensorCopy = np.clip(imageSortieTensorCopy,plageVal[0],plageVal[1])
    return [imageEntreeTensor,np.array(imageSortieTensorCopy,dtype=formatArray)]

In [0]:
def normalisation(arrayL,plageEntree=[0,255],plageSortie=[0,1]):
    assert plageEntree != plageSortie
    assert plageEntree[1]>0 and plageSortie[1] > 0
    formatArray = [array.dtype for array in arrayL]
    L = [np.array(array,dtype=np.float) for array in arrayL]
    for i in range(len(L)):
        L[i] = np.array((L[i]-plageEntree[0])/(plageEntree[1]-plageEntree[0])*(plageSortie[1]-plageSortie[0])+plageSortie[0],formatArray[i])
    return L

In [0]:
def LarrayFloatToUint(L):
    return [np.array(array,np.uint) for array in L]

In [0]:
def tauxApprentissage(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def differenceAcceptee(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def convertToUint(array):
    return np.array(normalisation(array,[0,1],[0,255]),dtype=np.uint8)

In [0]:
def convertToUintL(L):
    Lresult = []
    print("Entree : ",len(L))
    for i in range(len(L)):
        Lresult.append(np.array(normalisation(L[i],[0,1],[0,255]),dtype=np.uint8))
    print("Sortie : ",len(Lresult))
    return Lresult

In [0]:
index = np.arange(0,len(images))
np.random.shuffle(index)
trainingData = [images[i] for i in index[:int(0.6*len(images))]]
evalData = [images[i] for i in index[int(0.6*len(images))-1:]]

In [0]:
# source : https://stackoverflow.com/questions/46418373/how-to-resize-interpolate-a-tensor-in-keras
def interpolation(h,w,inputTensor):
    def resize_like(inputTensor,h,w):
        return tf.image.resize_nearest_neighbor(inputTensor, [h, w])

    return Lambda(resize_like, arguments={'h':h,'w':w})(inputTensor)

In [0]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
def SELU(x):
    return 1.0507*K.elu(x,alpha=1.67326)

get_custom_objects().update({'custom_activation': Activation(SELU)})

# A mettre pour le modèle : Activation(SELU)

In [0]:
class LRN2D(Layer):#Normalisation de réponse locale
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5, **kwargs):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__(**kwargs)
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = K.shape(X)
        half_n = self.n // 2
        input_sqr = K.square(X)
        extra_channels = K.zeros((b, ch + 2 * half_n, r, c))
        input_sqr = K.concatenate([extra_channels[:, :half_n, :, :],
                                   input_sqr,
                                   extra_channels[:, half_n + ch:, :, :]],
                                  axis=1)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i + ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        config = {"name": self.__class__.__name__,
                  "alpha": self.alpha,
                  "k": self.k,
                  "beta": self.beta,
                  "n": self.n}
        base_config = super(LRN2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
# cf https://stackoverflow.com/questions/43784921/how-to-display-custom-images-in-tensorboard-using-keras?noredirect=1#comment85726690_43784921
def make_image(tensor):
    """
    Convert an numpy representation image to Image protobuf.
    Copied from https://github.com/lanpa/tensorboard-pytorch/
    """
    from PIL import Image
    tensor = np.stack((tensor,tensor,tensor),axis=-1)
    height, width, channel = tensor.shape # numpy.ndarray
    image = Image.fromarray(tensor)
    import io
    output = io.BytesIO()
    image.save(output, format='PNG')
    image_string = output.getvalue()
    output.close()
    CHANNEL = 1
    var = tf.Summary.Image(height=height,
                         width=width,
                         colorspace=CHANNEL,
                         encoded_image_string=image_string)
    print("var : ",var)
    return var

In [0]:
def print_shape(x):
    print("Shape : ",x.get_shape().as_list())

In [0]:
index_couches = 0

In [0]:
from graphviz import render
from graphviz import Digraph,Graph
def g_get_current_id():
	global index_couches
	return str(index_couches)
def g_get_past_id(back=1):
	global index_couches
	return str(index_couches-back)
def g_new_id():
	global index_couches
	index_couches += 1
	return str(index_couches)
def g_link(graph,id1,id2):
	graph.edge(id1,id2)
def g_auto_link(graph):
	graph.edge(g_get_past_id(),g_get_current_id())

def g_conv(graph,prev,noyau, filtres, strides = 1, auto_connect = True, identifier=False):
	label = None
	if type(noyau) == int:
		label = "{Convolution | {Noyau | %d} | {Filtres | %d} | {Strides | %d}}"%(noyau,filtres,strides)
	else:
		label = "{Convolution | {Noyau | %dx%d} | {Filtres | %d} | {Strides | %d}}"%(noyau[0],noyau[1],filtres,strides)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def g_max_p(graph,prev,noyau, auto_connect = True, identifier=False):
	label = "{MaxPooling | {Noyau | %d}}"%(noyau)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def g_dense(graph,prev,filtres, strides = 1, auto_connect = True, identifier=False):
	label = "{Dense | {Filtres | %d} | {Strides | %d}}"%(filtres,strides)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())

def g_dropout(graph,prev,taux, auto_connect = True, identifier=False):
	label = ""
	if type(taux) == float:
		label = "{Dropout | {Rate\n(taux désactivation) | %.3f}}"%(taux)
	else:
		label = "{Dropout | {Rate\n(taux désactivation) | Adapté}}"

	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def g_regLoc(graph,prev,noyau = 20, k = 2, alpha= 10**-4,beta = 0.75, auto_connect = True, identifier=False):
	label = "{Regularisation\nRéponse\nLocale | {Noyau | %d} | {k | %.2f} | {alpha | %.2e} | {beta | %.2e}}"%(noyau,k,alpha,beta)
	if identifier == True:
		print(identifier)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def g_activation(graph,prev,type = "SELU", auto_connect = True, identifier=False):
	label = "{Activation | {Type | %s}}"%(type)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def g_batch_norm(graph,prev, auto_connect = True, identifier=False):
	label = "{Normalisation\nPar\nBatch}"
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def g_flat(graph,prev, auto_connect = True, identifier=False):
	label = "{Flatten}"
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white" if identifier==False else "red",style="filled")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def g_concat(graph,index, identifier=False):
    graph.node(g_get_current_id(),"Concatenate",fillcolor="white" if identifier==False else "red")
    for id in index:
        graph.edge(id,g_get_current_id())
def g_add(graph,index, identifier=False):
    graph.node(g_get_current_id(),"+",fillcolor="white" if identifier==False else "red")
    for id in index:
        graph.edge(id,g_get_current_id())
def g_subtract(graph,index, identifier=False):
    graph.node(g_get_current_id(),"-",fillcolor="white" if identifier==False else "red")
    for id in index:
        graph.edge(id,g_get_current_id())
def g_proba(graph,prev, auto_connect = True, identifier=False):
	graph.node(g_get_current_id(),"Probabilite",fillcolor="white" if identifier==False else "red")
	if auto_connect == True:
		g_link(graph,id1=prev,id2=g_get_current_id())
def end_graph(graph):
	graph.render("Modele000001")

In [0]:
def extract_inpt(inpt):
    return inpt[0],inpt[1],inpt[2],inpt[3]
def conv(inpt,k,f,s,operation=False, identifier=False,draw=True):
    inpt,graph,index,improvements = extract_inpt(inpt)
    global index_couches
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    couche = None
    trainable = True if improvements == None or max(improvements-1,0) == 1 else False
    if type(k) == int: ## Modif à vérifier
        couche = Convolution2D(filters=f,kernel_size=(k,k),activation=None,strides=(s,s),padding='SAME',name="%d_conv_k%d_f%d"%(index_couches,k,f),trainable=trainable)(inpt)
    else:#Otherwise we consider that it is a tuple
        couche = Convolution2D(filters=f,kernel_size=k,activation=None,strides=(s,s),padding='SAME',name="%d_conv_k%d_%d_f%d"%(index_couches,k[0],k[1],f),trainable=trainable)(inpt)
    if draw == True:
        if operation == False:
            g_conv(graph,prev=index,noyau=k,filtres=f,strides=s, identifier=identifier)
        else:
            g_conv(graph,prev=index,noyau=k,filtres=f,strides=s,auto_connect=False, identifier=identifier)
    if identifier == True:
        print("Previous index %s current %s"%(index,g_get_current_id()))
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-2,0)]## Modif


### To modify
def dropout(inpt,r,operation=False, identifier=False,draw=True):
    name = ""
    global index_couches
    if type(r) != float:
        r = K.cast(r[0,0],K.floatx())
        name = '%d_dropout_r_adaptative'%(index_couches)
    else:
        name = '%d_dropout_r%.2f'%(index_couches,r)
    inpt,graph,index,improvements = extract_inpt(inpt)
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    trainable = True if improvements == None or max(improvements-1,0) == 1 else False
    couche = Dropout(name=name,rate=r,trainable=trainable)(inpt)
    if draw == True:
        if operation == False:
            g_dropout(graph,prev=index,taux=r, identifier=identifier)
        else:
            g_dropout(graph,prev=index,taux=r,auto_connect=False, identifier=identifier)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-2,0)]
def lrn(inpt,n,k,a,b,operation=False, identifier=False,draw=True):
    inpt,graph,index,improvements = extract_inpt(inpt)
    global index_couches
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    couche = LRN2D(name='%d_lrn_n%d_k%d_a%.2e_b%.2f'%(index_couches,n,k,a,b),n=n,k=k,alpha=a,beta=b)(inpt)
    if draw == True:
        if operation == False:
            g_regLoc(graph,prev=index, identifier=identifier)
        else:
            g_regLoc(graph,prev=index,auto_connect=False, identifier=identifier)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-1,0)]
def activ(inpt,act_type,operation=False, identifier=False,draw=True):
    inpt,graph,index,improvements = extract_inpt(inpt)
    global index_couches
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    assert act_type=="SELU", "other activation functions than SELU are currently not supported"
    couche = Activation(SELU,name='%d_activation_%s'%(index_couches,act_type))(inpt)
    if draw == True:
        if operation == False:
            g_activation(graph,prev=index, identifier=identifier)
        else:
            g_activation(graph,prev=index,auto_connect=False, identifier=identifier)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-1,0)]
def b_norm(inpt,operation=False, identifier=False,draw=True):
    inpt,graph,index,improvements = extract_inpt(inpt)
    global index_couches
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    trainable = True if improvements == None or max(improvements-1,0) == 1 else False
    couche = BatchNormalization(name='%d_batchnorm'%(index_couches),trainable=trainable)(inpt)
    if draw == True:
        if operation == False:
            g_batch_norm(graph,prev=index, identifier=identifier)
        else:
            g_batch_norm(graph,prev=index,auto_connect=False, identifier=identifier)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-2,0)]
def dense(inpt,f,operation=False, identifier=False,draw=True):
    global index_couches
    inpt,graph,index,improvements = extract_inpt(inpt)
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    trainable = True if improvements == None or max(improvements-1,0) == 1 else False
    couche = Dense(f,activation=None,name='%d_dense_f%d'%(index_couches,f),trainable=trainable)(inpt)
    if draw == True:
        if operation == False:
            g_dense(graph,prev=index,filtres=f, identifier=identifier)
        else:
            g_dense(graph,prev=index,filtres=f,auto_connect=False, identifier=identifier)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-2,0)]
def extract_inpt_L(L):
    L_inpt = [L[i][0] for i in range(len(L))]
    graph = L[0][1]
    ids = [inpt[2] for inpt in L]

    return L_inpt,graph,ids,L[-1][3]
def concat(L_inpt,operation=False, identifier=False,draw=True):
    L_inpt,graph,index,improvements = extract_inpt_L(L_inpt)
    global index_couches
    couche = Concatenate(axis=-1,name='%d_merge'%(index_couches))(L_inpt)
    if draw == True:
        if operation == False:
            g_concat(graph,index, identifier=identifier)
        else:
            g_concat(graph,index, auto_connect=False, identifier=identifier)

    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-1,0)]
def subtract(L_inpt,operation=False, identifier=False):
    L_inpt,graph,index,improvements = extract_inpt_L(L_inpt)
    global index_couches
    couche = Subtract(name='%d_subtract'%(index_couches))(L_inpt)
    if draw == True:
        if operation == False:
            g_subtract(graph,index, identifier=identifier)
        else:
            g_subtract(graph,index, auto_connect=False, identifier=identifier)

    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-1,0)]
def max_p(inpt,k,operation=False, identifier=False,draw=True):
    inpt,graph,index,improvements = extract_inpt(inpt)
    global index_couches
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    trainable = True if improvements == None or max(improvements-1,0) == 1 else False
    couche = MaxPooling2D(name='%d_max_p_k%d'%(index_couches,k),pool_size=k,padding='VALID',trainable=trainable)(inpt)
    if draw == True:
        if operation == False:
            g_max_p(graph,prev=index,noyau=k, identifier=identifier)
        else:
            g_max_p(graph,prev=index,noyau=k,auto_connect=False, identifier=identifier)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-2,0)]
def add(L_inpt,operation=False, identifier=False,draw=True):
    global index_couches
    if L_inpt[0][0].get_shape().as_list()[-1] != 3:
        if L_inpt[0][3] != None:
            L_inpt[0][3] = max(L_inpt[0][3],1)
        L_inpt[0] = conv(L_inpt[0],k=1,f=3,s=1)
    if L_inpt[1][0].get_shape().as_list()[-1] != 3:
        if L_inpt[1][3] != None:
            L_inpt[1][3] = max(L_inpt[1][3],1)
        L_inpt[1] = conv(L_inpt[1],k=1,f=3,s=1)
    L_inpt,graph,index,improvements = extract_inpt_L(L_inpt)
    couche = Add(name='%d_add'%(index_couches))(L_inpt) 
    if draw == True:
        if operation == False:
            g_add(graph,index,identifier=identifier)
        else:
            g_add(graph,index,auto_connect=False,identifier=identifier)

    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-1,0)]
def flat(inpt,draw=True):
    inpt,graph,index,improvements = extract_inpt(inpt)
    global index_couches
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    couche = Flatten(name='%d_flatten'%(index_couches))(inpt)
    if draw == True:
        g_flat(graph,prev=index)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-1,0)]
def proba(inpt,draw=True):
    inpt,graph,index,improvements = extract_inpt(inpt)
    global index_couches
    if improvements != None and improvements == 0:
        index_couches += 1
        return [inpt,graph,g_get_past_id(),0]
    couche = Activation('sigmoid',name='%d_sigmoid_proba'%(index_couches))(inpt)
    if draw == True:
        g_proba(graph,prev=index)
    index_couches += 1
    return [couche,graph,g_get_past_id(),None if improvements == None else max(improvements-1,0)]

In [0]:
def magn_angle(x,draw=True):
    x,graph,index,improvements = x[0],x[1],x[2],x[3]
    x = Lambda(lambda x:K.cast(x,"complex64"))(x)
    x_list_magn = []
    x_list_angle = []
    for i in range(3):
        fft = Lambda(lambda x: tf.fft2d(x[:,:,:,i]), output_shape=(None,199,199))(x)
        x_list_magn.append(Lambda(lambda fft:K.expand_dims(tf.math.abs(fft),axis=-1), output_shape=(None,199,199))(fft))
        x_list_angle.append(Lambda(lambda fft: K.expand_dims(tf.math.angle(fft),axis=-1), output_shape=(None,199,199))(fft))
    magn = Concatenate()(x_list_magn)
    angle = Concatenate()(x_list_angle)
    magn = Lambda(lambda magn: K.cast(magn,dtype=tf.float32), output_shape=(None,199,199))(magn)
    angle = Lambda(lambda angle: K.cast(angle,dtype=tf.float32), output_shape=(None,199,199))(angle)
    if draw == True:
        graph.node(g_get_current_id(),"FFT")
        g_link(graph,g_get_past_id(),g_get_current_id())
    global index_couches
    index_couches += 1
    return [magn,graph,g_get_past_id(),improvements],[angle,graph,g_get_past_id(),improvements]
def to_rgb(magn,angle,draw=True):
    magn,graph,magn_index,improvements = extract_inpt(magn)
    angle,graph,angle_index,improvements = extract_inpt(angle)
    magn = Lambda(lambda magn: K.cast(magn,dtype=tf.complex64))(magn)
    angle = Lambda(lambda angle:K.cast(angle,dtype=tf.complex64))(angle)
    x_list_canal = []
    for i in range(3):
        complx_tensor = Lambda(lambda mang_angle: mang_angle[0][:,:,:,i]*tf.exp(1j*mang_angle[1][:,:,:,i]))([magn,angle])
        ifft = Lambda(tf.ifft2d)(complx_tensor)
        x_list_canal.append(K.expand_dims(ifft,axis=-1))
    images = Concatenate()(x_list_canal)
    images = K.cast(images,dtype=tf.float32)
    if draw == True:
        graph.node(g_get_current_id(),"RGB")
        g_link(graph,magn_index,g_get_current_id())
        g_link(graph,angle_index,g_get_current_id())
    global index_couches
    index_couches += 1
    return [images,graph,g_get_past_id(),improvements]

In [0]:
def unit(inpt,k,f,dropout_r=None,activation=False,draw=False,init_index=None):
    if init_index != None:
        global index_couches
        index_couches = init_index
    if inpt[3] != None:
        activation = True
    couche = conv(inpt,k=(1,k),f=f,s=1,draw=draw)
    couche = conv(couche,k=(k,1),f=f,s=1,draw=draw)
    couche = b_norm(couche,draw=draw)
    couche = lrn(couche,n=21,k=2,a=10**-4,b=0.75,draw=draw)
    if activation == True:
        couche = activ(couche,act_type="SELU",draw=draw)
    if dropout_r != None:
        couche = dropout(couche,r=dropout_r,draw=draw)
    return couche

In [0]:
def inception(inpt,draw=True):
    orig_graph = inpt[1]
    inpt[1] = Digraph(name="cluster_Inception%s"%(g_get_current_id()), node_attr={'style':'filled', 'color':'white'})
    inpt[1].attr(style='filled', color='red', label="Inception")
    
    T0 = unit(inpt,k=1,f=100,activation=True,dropout_r=0.5,draw=draw)
    T1 = unit(inpt,k=1,f=100,activation=True,dropout_r=0.5,draw=draw)
    T1 = unit(T1,k=1,f=100,activation=True,dropout_r=0.5,draw=draw)


    couche = concat([T0,T1],draw=draw)
    couche = conv(couche,k=1,f=100,s=1,draw=draw)
    couche = add([couche,inpt],draw=draw)
    orig_graph.subgraph(couche[1])
    couche[1] = orig_graph
    return couche
def analyse(couche,dropout_rate,draw,type_donnee):
    orig_graph = couche[1]
    couche[1] = Digraph(name="cluster_Analyse_%s"%(type_donnee), node_attr={'style':'filled', 'color':'white'})
    couche[1].attr(style='filled', color='orange',label="Analyse_%s"%(type_donnee))
    global index_couches
    #index 2 and 18
    T0 = unit(couche,k=2,f=100,activation=True,dropout_r=0.25)
    
    T1 = unit(couche,k=2,f=50,activation=True,dropout_r=0.25)
    T1 = unit(T1,k=2,f=50,activation=True,dropout_r=0.25)
    
    T_straight = dropout(couche,r=dropout_rate,draw=draw)
    couche = concat([T0,T1],draw=draw)
    couche = conv(couche,k=1,f=3,s=1,draw=draw)
    couche = add([T_straight,couche],draw=draw)
    orig_graph.subgraph(couche[1])
    couche[1] = orig_graph
    return couche

In [0]:
def analyse_progressive_training_model(couche,step=0,init=2):
    draw = False
    T0 = unit(couche,k=2,f=100,activation=True,dropout_r=0.25)
    T1 = unit(couche,k=2,f=50,activation=True,dropout_r=0.25)
    T1 = unit(T1,k=2,f=50,activation=True,dropout_r=0.25)
    T_straight = dropout(couche,r=0.2,draw=draw)
    couche_concat = concat([T0,T1],draw=draw)
    couche = conv(couche_concat,k=1,f=3,s=1,draw=draw)
    couche = add([T_straight,couche],draw=draw)
    return  couche


In [0]:
def model_fft_training(step,half_batch_size,magn_select=True):#else angle
    global index_couches
    global index_disc
    index_couches = 1
    inpt = Input(shape = (199, 199, 3),name=str(0)+'_'+'Image')
    inpt_real = Input(shape = (199, 199, 3),name=str(0)+'_'+'Image_real')
    graph = Digraph(name='cluster_Graph', format='png')
    graph.attr(bgcolor='transparent')
    graph.node("0","Image")
    inpt = [inpt,graph,g_get_past_id(),step]
    inpt_real = [inpt_real,graph,g_get_past_id(),step]
    magn,angle = magn_angle(inpt,draw=False)
    r_magn,r_angle = magn_angle(inpt_real,draw=False)

    inpt_analyse = magn if magn_select == True else angle
    img_layer = analyse_progressive_training_model(inpt_analyse,step=step,init=2 if magn_select == True else 18)#In order to keep original indexes
    
    img_rgb = to_rgb(img_layer if magn_select == True else r_angle,img_layer if magn_select == False else r_magn) 
    model = Model(inputs=[inpt[0],inpt_real[0]],outputs=img_rgb[0])
    return model

In [0]:
nb_couches = 3
index_disc = 0
def AI_gen_disc_or_disc_without_input(inpt,dropout_rate=None,part='gen_disc',draw=True):
    global index_couches
    global index_disc
    index_couches = 1
    inpt = Lambda(lambda x:K.cast(x,dtype=tf.float32))(inpt)
    graph = Digraph(name='cluster_Graph', format='png')
    graph.attr(bgcolor='transparent')
    graph.node("0","Image")
    inpt = [inpt,graph,g_get_past_id()]
    if part == 'gen_disc' or part == 'gen':
        gen_graph = Digraph(name="cluster_Generator", node_attr={'style':'filled', 'color':'white'})
        gen_graph.attr(style='filled', color='darkgreen',label="Generator")
        inpt[1] = gen_graph
        with K.name_scope('Generateur_FFT'):
            inpt[1] = Digraph(name="cluster_Analyse_FFT", node_attr={'style':'filled', 'color':'white'})
            inpt[1].attr(style='filled', color='lightgrey', label="Analyse_FFT")
            magn,angle = magn_angle(inpt,draw=draw)
            
            
            magn = analyse(magn,dropout_rate,draw=draw,type_donnee="magn")
            angle = analyse(angle,dropout_rate,draw=draw,type_donnee="angle")

            gen_graph.subgraph(angle[1])
            magn[1] = gen_graph
            angle[1] = gen_graph
            fft_back = to_rgb(magn,angle,draw=draw)
        with K.name_scope('Generateur_image'):
            inpt[1] = Digraph(name="cluster_Analyse_Image", node_attr={'style':'filled', 'color':'white'})
            inpt[1].attr(style='filled', color='blue',label="Analyse_image")
            couche = conv(inpt,k=2,f=200,s=1,draw=draw)
            couche = dropout(couche,r=0.25,draw=draw)
            couche = lrn(couche,n=21,k=2,a=10**-4,b=0.75,draw=draw)
            couche = activ(couche,act_type="SELU",draw=draw)
            couche = conv(couche,k=2,f=100,s=1,draw=draw)
            couche = dropout(couche,r=0.25,draw=draw)
            couche = b_norm(couche,draw=draw)
            couche = activ(couche,act_type="SELU",draw=draw)
            couche = conv(couche,k=2,f=100,s=1,draw=draw)
            couche = dropout(couche,r=0.25,draw=draw)
            couche = b_norm(couche,draw=draw)
            couche = activ(couche,act_type="SELU",draw=draw)
            gen_graph.subgraph(couche[1])
            
            couche[1] = gen_graph
            couche = concat([couche,fft_back],draw=draw)
            couche = conv(couche,k=2,f=100,s=1,draw=draw)
            couche = dropout(couche,r=0.25,draw=draw)
            couche = b_norm(couche,draw=draw)
            couche = activ(couche,act_type="SELU",draw=draw)
            couche = conv(couche,k=2,f=100,s=1,draw=draw)
            couche = dropout(couche,r=0.25,draw=draw)
            couche = b_norm(couche,draw=draw)
            couche = activ(couche,act_type="SELU",draw=draw)
            couche = dense(couche,f=3,draw=draw)
            couche = b_norm(couche,draw=draw)
            couche = activ(couche,act_type="SELU",draw=draw)
            index_disc = index_couches
            if part == 'gen':
                return couche[0]
        graph.subgraph(gen_graph)
    if part == 'gen_disc' or part == 'disc':
        disc_graph = Digraph(name="cluster_Discriminator", node_attr={'style':'filled', 'color':'white'})
        disc_graph.attr(style='filled', color='darkorange1',label="Discriminator")
        with K.name_scope('Discriminateur'):
            index_couches = index_disc
            if part == 'gen_disc':
                couche[1] = disc_graph
                coucheAdaptation = conv(couche,k=2,f=100,s=1,draw=draw)
            else:
                inpt[1] = disc_graph
                coucheAdaptation = conv(inpt,k=2,f=100,s=1,draw=draw)
            image = inception(coucheAdaptation,draw=draw)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75,draw=draw)
            image = b_norm(image,draw=draw)
            image = activ(image,act_type="SELU",draw=draw)
            image = max_p(image,k=3,draw=draw)
            image = inception(image,draw=draw)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75,draw=draw)
            image = b_norm(image,draw=draw)
            image = activ(image,act_type="SELU",draw=draw)
            image = max_p(image,k=3,draw=draw)
            image = inception(image,draw=draw)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75,draw=draw)
            image = b_norm(image,draw=draw)
            image = activ(image,act_type="SELU",draw=draw)
            image = inception(image,draw=draw)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75,draw=draw)
            image = b_norm(image,draw=draw)
            image = activ(image,act_type="SELU",draw=draw)
            image = max_p(image,k=3,operation=False,draw=draw)
        
            resultatAnalyse = flat(image,draw=draw)
            resultatAnalyse = dropout(resultatAnalyse,r=0.1,draw=draw)
            resultatAnalyse = dense(resultatAnalyse,f=10,draw=draw)
            resultatAnalyse = dropout(resultatAnalyse,r=0.25,draw=draw)
            resultatAnalyse = activ(resultatAnalyse,act_type="SELU",draw=draw)
            resultatAnalyse = dense(resultatAnalyse,f=1,draw=draw)
            probabilite = proba(resultatAnalyse,draw=draw)
        graph.subgraph(probabilite[1])
        end_graph(graph)
        return probabilite[0]

In [0]:
def AI():
    inpt = Input(shape = (199, 199, nb_couches),name=str(0)+'_'+'Image')
    inpt_dropout = Input(shape = (1),name=str(0)+'_'+'Dropout_rate')
    probabilite_gen_disc = AI_gen_disc_or_disc_without_input(inpt,inpt_dropout,part='gen_disc',draw=True)
    probabilite_disc = AI_gen_disc_or_disc_without_input(inpt,part='disc',draw=False)
    generated_image = AI_gen_disc_or_disc_without_input(inpt,inpt_dropout,part='gen',draw=False)
    return Model(inputs=[inpt,inpt_dropout],outputs=probabilite_gen_disc),Model(inputs=[inpt,inpt_dropout],outputs=probabilite_disc), Model(inputs=[inpt,inpt_dropout],outputs=generated_image)

In [0]:
def mean_gen_disc_top_27(disc,gen,gen_disc,input_image,desired_image,indexModele):
    path_model = "Models/Modele"+str(indexModele)+"_"
    partial_path_gen_backup = path_model + 'gen' + '_'
    partial_path_gen_disc_backup = path_model + 'gen_disc' + '_'
    partial_path_disc_backup = path_model + 'disc' + '_'

    Lloss = []
    Lloss_gen = []
    Lloss_gen_disc = []
    Lloss_disc = []
    
    try:
        gen.save_weights(partial_path_gen_backup + 'orig.h5')
    except:
        gen.save_weights(partial_path_gen_backup + 'orig.h5')
    try:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    except:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    try:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    except:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')

    for i in range(3):
        gen_backup = partial_path_gen_backup + str(i) + '.h5'
        print("Training "+gen_backup)
        try:
            gen.load_weights(partial_path_gen_backup + 'orig.h5')
        except:
            gen.load_weights(partial_path_gen_backup + 'orig.h5')
        loss = gen.train_on_batch(input_image,desired_image)
        try:
            gen.save_weights(gen_backup)
        except:
            gen.save_weights(gen_backup)
        Lloss.append([gen_backup,loss])
    try:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    except:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    try:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    except:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    print("Lloss : ",Lloss)
    desired_probability = [1]*input_image[0].shape[0] # Batch size
    Lloss_gen_disc_tmp = []
    for index_gen_disc,path_loss_backup in enumerate(Lloss):
        try:
            gen.load_weights(path_loss_backup[0])
        except:
            gen.load_weights(path_loss_backup[0])
        for i in range(3):
            gen_disc_backup = partial_path_gen_disc_backup + str(i) + '.h5'
            print("Training "+gen_disc_backup)
            try:
                gen_disc.load_weights(partial_path_gen_disc_backup + 'orig.h5')
            except:
                gen_disc.load_weights(partial_path_gen_disc_backup + 'orig.h5')
            loss = gen_disc.train_on_batch(input_image,desired_probability)
            try:
                gen_disc.save_weights(gen_disc_backup)
            except:
                gen_disc.save_weights(gen_disc_backup)
            Lloss_gen_disc_tmp.append(path_loss_backup + [gen_disc_backup,loss])
    Lloss = Lloss_gen_disc_tmp
    print("Lloss : ",Lloss)

    desired_probability = [1]*(input_image[0].shape[0]//2)+[0]*(input_image[0].shape[0]//2)
    Lloss_disc_tmp = []
    for index_model,item in enumerate(Lloss):
        for i in range(3):
            disc_backup = partial_path_disc_backup + str(i) + '.h5'
            print("Training "+disc_backup)
            try:
                gen_disc.load_weights(Lloss[index_model][-2])
            except:
                gen_disc.load_weights(Lloss[index_model][-2])
            loss = disc.train_on_batch(input_image,desired_probability)
            try:
                disc.save_weights(disc_backup)
            except:
                disc.save_weights(disc_backup)
            Lloss_disc_tmp.append(Lloss[index_model]+[disc_backup,loss])


    Lloss = [Lloss_disc_tmp[i] + [(Lloss_disc_tmp[i][1]+Lloss_disc_tmp[i][3]+Lloss_disc_tmp[i][5])/3] for i in range(len(Lloss_disc_tmp))]
    print("Lloss : ",Lloss)
    Lloss.sort(key=lambda elem:elem[-1])
    print("Lloss sorted : ",Lloss)
    try:
        gen_disc.load_weights(Lloss[0][2])
        disc.load_weights(Lloss[0][-3])
    except:
        gen_disc.load_weights(Lloss[0][2])
        disc.load_weights(Lloss[0][-3])
    print("Best model back")
    return Lloss[0]

In [0]:
def sauvegardeModele(entree_pure,entree_deterioree,model,iteration_entrainement,summary_writer,batch_size=5):
    global Llayers
    for canal_image in range(3):
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_pure_canal_"+str(canal_image), 
                                image=make_image(np.array(entree_pure[0][0,:,:,canal_image]*255,dtype=np.uint8)))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_deterioree_canal_"+str(canal_image), 
                                image=make_image(np.array(entree_deterioree[0][0,:,:,canal_image]*255,dtype=np.uint8)))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        
    for p,entree in enumerate([entree_pure,entree_deterioree]):
        print("p value %d"%(p))
        layer_outputs,layer_names = [Llayers[i][0] for i in range(len(Llayers))],[Llayers[i][1] for i in range(len(Llayers))]
        model_calcul_image = Model(inputs=model.input,outputs=[model.output]+layer_outputs)
        sorties_couches = model_calcul_image.predict(entree, batch_size=batch_size)[1:] if len(layer_outputs) > 0 else [model_calcul_image.predict(entree, batch_size=batch_size)][1:]
        
        for index_couche,sortie_couche in enumerate(sorties_couches):
            layer_name = layer_names[index_couche]
            dim_sortie = sortie_couche.shape
            if len(dim_sortie) == 4:
                for canal_image in range(dim_sortie[-1]):
                    tag = layer_name
                    tag += 'pure' if p == 0 else 'deterioree'
                    tag += "_canal_" + str(canal_image)
                    summary_image = tf.Summary(value=[tf.Summary.Value(tag=tag, 
                                            image=make_image(np.array(sortie_couche[0,:,:,canal_image]*255,dtype=np.uint8)))])
                    summary_writer.add_summary(summary_image,iteration_entrainement)
    return None

In [0]:
from google.colab import output
def beep():
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
    return None

In [0]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
continuer = None
def block_or_continue():
    beep()
    print("Would you resume training ? True or False ?")
    return bool(input())

In [0]:
def backup(image,imageBruitee,gen_disc,disc,index,summary_writer):
    x = [np.concatenate((image[0],imageBruitee[0])),np.concatenate((image[1],imageBruitee[1]))]
    sauvegardeModele(image,imageBruitee,gen_disc,index,summary_writer)
    p_gen_disc = gen_disc.predict(x)[0]
    p_disc = disc.predict(x)[0]

    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig pure) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig pure) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)
    x_proba = [np.concatenate((imageBruitee[0],imageBruitee[0])),np.concatenate((imageBruitee[1],imageBruitee[1]))]
    p_gen_disc = gen_disc.predict(x_proba)[0]
    p_disc = disc.predict(x_proba)[0]
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)

In [0]:
def custom_loss(y_true,y_pred):
    return K.mean(K.square(y_pred - y_true),axis=-1)+K.max(K.square(y_pred - y_true),axis=-1)

In [0]:
def to_fft(img_tensor):
    L = []
    print(img_tensor.shape[0],img_tensor.get_shape().as_list())
    for index_image in range(img_tensor.get_shape().as_list()[0]):
        L.append(to_fft_single(img_tensor[index_image,:,:,:]))
    return np.stack(L,axis=0)

def to_img(fft_cplx):
    return to_uint8_array(np.fft.ifft2(fft_cplx))
def to_fft_single(img):
    fft_cplx = np.fft.fft2(img)
    fft = 20*np.log(np.abs(fft_cplx)+10**-5*np.ones(img.shape))
    fft = fft + np.abs(np.min(fft))*np.ones(fft.shape)
    fft = fft / (np.max(fft)*np.ones(fft.shape))*(255*np.ones(fft.shape))
    fft_uint8 = to_uint8_array(fft)
    print(np.max(fft_uint8),np.min(fft_uint8),fft_uint8.shape,fft_uint8.dtype)
    return fft_cplx

def to_uint8_array(array):
    return array.astype(np.uint8)
def next_batch_bruit_voile_fft():
    image,imageBruitee = next_batch_bruit_voile_2(10,images,199,np.float32,[1,1,1],[50,50])
    

In [0]:
def train_progressive_model(index_model):
    max_step, max_it = 200,1000
    for step in range(1,max_step):
        for index_choix,select in enumerate([True,False]):
            model = model_fft_training(step=step,half_batch_size=5,magn_select=select)
            model.compile(loss=custom_loss,optimizer="Adam")
            import os
            if os.path.exists("Pretrain_fft_%s_model%d.h5"%("magn" if select == True else "angle",index_model)):
                model.load_weights("Pretrain_fft_%s_model%d.h5"%("magn" if select == True else "angle",index_model),by_name=True)
            for iteration in range(max_it):
                image,imageBruitee = next_batch_bruit_voile_2(5,images,199,np.float32,[1,1,1],[50,50])
                x = np.concatenate((image,imageBruitee),axis=0)
                y = np.concatenate((image,image),axis=0)
                loss = model.train_on_batch([x,y],y)
                if iteration % 5 == 0:
                    model.save_weights("Pretrain_fft_%s_model%d.h5"%("magn" if select == True else "angle",index_model))
                print("Etape d'entrainement %d de %s iteration %d statut %d pourcent, erreur %.5f"%(step,"magn" if select == True else "angle", iteration,int(((step-1)*2*max_it+index_choix*max_it+iteration)/((max_step-1)*2*max_it)*100),loss))

In [0]:
train_progressive_model(10)

Etape d'entrainement 1 de magn iteration 0 statut 0 pourcent, erreur 8307.76953
Etape d'entrainement 1 de magn iteration 1 statut 0 pourcent, erreur 7072.95117
Etape d'entrainement 1 de magn iteration 2 statut 0 pourcent, erreur 7239.45752
Etape d'entrainement 1 de magn iteration 3 statut 0 pourcent, erreur 5453.95508
Etape d'entrainement 1 de magn iteration 4 statut 0 pourcent, erreur 6472.72998
Etape d'entrainement 1 de magn iteration 5 statut 0 pourcent, erreur 4880.99121
Etape d'entrainement 1 de magn iteration 6 statut 0 pourcent, erreur 8581.56738
Etape d'entrainement 1 de magn iteration 7 statut 0 pourcent, erreur 7972.54004
Etape d'entrainement 1 de magn iteration 8 statut 0 pourcent, erreur 7788.65234
Etape d'entrainement 1 de magn iteration 9 statut 0 pourcent, erreur 5523.43311
Etape d'entrainement 1 de magn iteration 10 statut 0 pourcent, erreur 13552.23535
Etape d'entrainement 1 de magn iteration 11 statut 0 pourcent, erreur 5093.41113
Etape d'entrainement 1 de magn iterat

In [0]:
index_couches = 0
Llayers = []
def train_gen_disc(images,eval_data,indexModele):
    gen_disc,disc,gen = AI()
    raise Patate
    gen_disc_optim = SGD(lr=0.01,momentum=0.0,nesterov=True,name="gen_disc_optim")
    disc_optim = SGD(lr=0.01,momentum=0.0,nesterov=True,name="disc_optim")
    gen_optim = SGD(lr=0.01,momentum=0.0,nesterov=True,name='gen_optim')
    disc.compile(loss='binary_crossentropy', optimizer=disc_optim)
    gen_disc.compile(loss='binary_crossentropy', optimizer=gen_disc_optim)
    gen.compile(loss=custom_loss, optimizer=gen_optim)
    print("Layers : ",Llayers)
    print(gen_disc.summary())
    Lloss = []
    iterations = 200
    index = 0
    import datetime
    chaine_date = datetime.datetime.today().strftime('%Y-%m-%d_%Hh%Mmin%Ss') #cf http://strftime.org/ et https://www.science-emergence.com/Articles/Obtenir-la-date-daujourdhui-au-format-YYYY-MM-DD-avec-python/
    summary_writer = tf.summary.FileWriter(logdir='./logs/Modele'+str(indexModele)+'_gen_disc_'+chaine_date,graph=tf.get_default_graph())
    while index < iterations:
        image,imageBruitee = next_batch_bruit_voile_2(5,images,199,np.float32,[1,1,1],[50,50])
        # image,imageBruitee = to_fft_single(image),to_fft_single(imageBruitee)
        current_learning_rate = tauxApprentissage(index,10**-4,10,10**-7)

        # K.set_value(disc.optimizer.lr, current_learning_rate)
        # K.set_value(gen_disc.optimizer.lr, current_learning_rate)
        current_learning_rate = tauxApprentissage(index,2,25,10**-7)
        # K.set_value(gen.optimizer.lr, current_learning_rate)
        dropout_rate = np.array(np.clip(np.exp(index/iterations)-1,0,1),dtype=np.float32)
        dropout_rate = np.stack([dropout_rate for i in range(5*2)],axis=0)
        print(dropout_rate.shape)
        print("With dropout rate %.2f"%(dropout_rate[0]))
        loss = mean_gen_disc_top_27(disc=disc,gen=gen,gen_disc=gen_disc,input_image=[np.concatenate((image,imageBruitee)),dropout_rate],desired_image=np.concatenate((image,image)),indexModele=indexModele)
        Lloss.append(loss)
        print("Iteration %i : gen, erreur : %f, gen_disc, erreur : %f ; disc, erreur : %f ; erreur moyenne : %f"%(index,loss[1],loss[3],loss[-2],loss[-1]))
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur gen : MSE", 
                                             simple_value=loss[1]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur gen-disc : binary-crossentropy", 
                                             simple_value=loss[3]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur disc : binary-crossentropy", 
                                             simple_value=loss[-2]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur moyenne : binary-crossentropy", 
                                             simple_value=loss[-1]) ])
        summary_writer.add_summary(summary_loss,index)
        
        if index % 5 == 0:
            backup([image,dropout_rate],[imageBruitee,dropout_rate],gen_disc,disc,index,summary_writer)
        index += 1
    summary_writer.close()
Lloss = train_gen_disc(images,evalData,9)
# See https://stackoverflow.com/questions/48242585/tf-gradients-wont-work-with-tf-assign-but-works-with